## C S 329E HW 5

# Crossvalidation and hyperparameter selection

## Michael Walters, Adrian Garcia - Goup 31

For this week's homework we are going to explore the cross validation testing methodology and applying that to get error estimates on the two algorithms we've used so far:
  - Linear Regression
  - Decision Tree classification
  

In [ ]:
# Google colab's default version of scikit-learn isn't the latest, so you will 
# need to update the virtual machine and restart the runtime
!pip install scikit-learn==1.0

  Using cached scikit_learn-1.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (23.1 MB)
  Using cached threadpoolctl-3.0.0-py3-none-any.whl (14 kB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
# import the libraries! If you want to add things here for visualization, please do, 
# but only use sklearn when prompted
import pandas as pd
import numpy as np
from sklearn import tree 
from sklearn.datasets import load_iris
from sklearn.datasets import load_diabetes

# Part 1 - Calculate Generalized Error on Linear Regression with k-fold Cross Validation

## Q1.1 Load in the diabetes data set as a pandas dataframe and series.  
Documentation on the data set is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html).  Name your features dataframe (the independent variables) `df_X` and your target value (the dependent variable) series `s_y`

In [ ]:
df_X, s_y = load_diabetes(as_frame=True, return_X_y=True)
df_X  

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641
...,...,...,...,...,...,...,...,...,...,...
437,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207
438,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018118,0.044485
439,0.041708,0.050680,-0.015906,0.017282,-0.037344,-0.013840,-0.024993,-0.011080,-0.046879,0.015491
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044528,-0.025930


## Q1.2 Define a function that creates a linear least squares regression model and a function to predict a continuous value given a linear regression model
The first function should take in two parameters, `df_X`, and `s_y` and return the least squares regression model, $\hat{\beta}$ (using the notation from the ESLII book chapter 3 and HW3).  You can not use any libraries outside of pandas and numpy. Note that the length of beta_hat should be the number of columns in `df_X` + 1. 

The second function should take in two parameters, `beta_hat` - the model generated from the `get_linear_regression` function, and `df_X` - that has the attribute values at which we want to predict a continuous value.  We assume that the format and ordering of `df_X` used to create the model and `df_X` used to generate predictions are consistent. 

In [ ]:
def get_linear_regression_model( df_X, s_y ):
  dfx = df_X.copy()
  ones = np.ones(len(dfx))
  dfx.insert(loc=0, column='intercept', value=ones)

  x_matrix = dfx.to_numpy()
  y_matrix = s_y.to_numpy()

  xtrans = np.transpose(x_matrix)
  x_pseudo = np.linalg.inv(np.matmul(xtrans,x_matrix))
  b_hat = np.matmul(np.matmul(x_pseudo,xtrans),y_matrix)
  return b_hat
  
get_linear_regression_model( df_X, s_y )


array([ 152.13348416,  -10.01219782, -239.81908937,  519.83978679,
        324.39042769, -792.18416163,  476.74583782,  101.04457032,
        177.06417623,  751.27932109,   67.62538639])

In [ ]:
# code to check beta_hat
np.random.seed(23)
beta_hat = get_linear_regression_model( pd.DataFrame(np.random.random((34,4))), pd.Series(np.random.random(34)*10.0) )
beta_hat

array([ 4.18818425,  1.77890808,  0.74032569, -1.3506416 ,  0.14535984])

In [ ]:
def predict_linear_regression_value( beta_hat, df_X ):
  dfx = df_X.copy()
  ones = np.ones(len(dfx))
  dfx.insert(loc=0, column='intercept', value=ones)

  x_matrix = dfx.to_numpy()
  return np.matmul(x_matrix, beta_hat)

In [ ]:
predicted_vals = predict_linear_regression_value( beta_hat, pd.DataFrame(np.random.random((3,4))))
predicted_vals

array([4.03669521, 4.39502589, 3.90884703])

## Q1.3 Define a function that partitions the dataframe and series data into dictionaries
This function should take in three parameters, `df_X`, `s_y`, and `k`, and returns a tuple of two dictionaries.
In both dictionaries the key is the `k` value (an integer from one to `k` inclusive).
The first dictionary will have the dataframe of the training data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable).
The second dictionary will have the series of the target data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable). Please note the targets _must match_ the same rows as the dataframe at this index, e.g, the length of the kth partition is the same for the dataframe and series.

Call that function with $k=5$ and create the dictionaries `dict_k_df_X` and `dict_k_s_y`. Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

Here is some example output from checking the length of the folds:
```
Fold 1 length of dataframe is 88 and length of series is 88
Fold 2 length of dataframe is 96 and length of series is 96
Fold 3 length of dataframe is 88 and length of series is 88
Fold 4 length of dataframe is 79 and length of series is 79
Fold 5 length of dataframe is 91 and length of series is 91
The sum of the number of elements in each fold is 442 and there are 442 rows in the original df
```

In [ ]:
def partition_data( df_X, s_y, k ):
  dfx = df_X.copy()
  dfx["bs"] = s_y
  rand_index = np.random.randint(low=1, high=5+1, size = len(dfx))

  dict_k_df_X = {}
  dict_k_s_y = {}

  for i in range(k):
    dft = dfx[rand_index == i+1]
    sy = dft["bs"]
    df_x = dft.drop(['bs'], axis = 1)

    dict_k_df_X[i+1] = df_x
    dict_k_s_y[i+1] = sy

  return (dict_k_df_X, dict_k_s_y)

In [ ]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5 )

In [ ]:
foldSum = 0
for key in (dict_k_df_X, dict_k_s_y)[0].keys():
  fold = key
  dfLength = len((dict_k_df_X, dict_k_s_y)[0][key])
  yLength = len((dict_k_df_X, dict_k_s_y)[1][key])
  foldSum += dfLength
  print(f"Fold {key} length of dataframe is {dfLength} and length of series is {yLength}")
print(f"The sum of the number of elements in each fold is {foldSum} and there are {len(df_X)} rows in the original df")

Fold 1 length of dataframe is 91 and length of series is 91
Fold 2 length of dataframe is 98 and length of series is 98
Fold 3 length of dataframe is 77 and length of series is 77
Fold 4 length of dataframe is 75 and length of series is 75
Fold 5 length of dataframe is 101 and length of series is 101
The sum of the number of elements in each fold is 442 and there are 442 rows in the original df


## Q1.4 Define a function that calculates a regression metric
This function should accept two series of equal length $n$ numpy arrays, `s_y`, and `s_y_hat`. The metric it should calculate is the mean absolute error, $MAE = \sum\limits_{i=1}^n\frac{|{s\_y_i - {s\_y\_hat}_i}|}{n}$ 

Test your function by using the vectors:
```
x = np.array([1,2,3])
y = np.array([2,2,3])
```


In [ ]:
def get_mae( s_y, s_y_hat):
    n = len(s_y)
    return (np.absolute(s_y - s_y_hat)/n).sum()

In [ ]:
# Test it 
x = np.array([1,2,3])
y = np.array([2,2,3])
get_mae(x,y)

0.3333333333333333

## Q1.5 Calculate the $MAE$ for each fold
For each fold in your dictionaries, calculate the $MAE$.  Use the partition number key as the test set, and all other partitions as the train set. 

Print the min, max, and mean $MAE$ of your 5 folds. 

You must use your helper functions that you wrote above, `get_linear_regression_model`, `predict_linear_regression_value` and `get_mae`. 

In [ ]:
mae = np.array([])
for k in dict_k_df_X.keys():
  dfXTest = dict_k_df_X[k]
  dfYTest = dict_k_s_y[k]

  dfXTrain = pd.DataFrame()
  dfYTrain = pd.DataFrame()

  for key in dict_k_df_X.keys():
    if key!= k:
      dfXTrain = pd.concat([dfXTrain, dict_k_df_X[key]])
      dfYTrain = pd.concat([dfYTrain, dict_k_s_y[key]])
    continue
    
  b_hat = get_linear_regression_model(dfXTrain, dfYTrain)
  s_y_hat = predict_linear_regression_value(b_hat, dfXTest)
  
  mae = np.append( mae, get_mae(dfYTest.to_numpy(), s_y_hat) )
mae

array([6555.51942129, 8088.41578701, 5751.60961629, 5811.20702516,
       7923.51452904])

In [ ]:
print("The min MAE is {:.2f}, the max MAE is {:.2f}, and the mean MAE is {:.2f}".format(mae.min(),mae.max(),mae.mean()))

The min MAE is 5751.61, the max MAE is 8088.42, and the mean MAE is 6826.05


# Part 2 - Find the best hyperparameter to use in a Decision Tree 

## Q2.1 Load the iris data in as a pandas dataframe and a series
Documentation on the data set is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). Name your features dataframe (the independent variables) `df_X` and your class label (the dependent variable) series `s_y`

In [ ]:
df_X, s_y = load_iris(as_frame=True, return_X_y=True)
s_y

0      0
1      0
2      0
3      0
4      0
      ..
145    2
146    2
147    2
148    2
149    2
Name: target, Length: 150, dtype: int64

## Q2.2 Partition `df_X` and `s_y` into $5$ partitions of roughly equal size
Make 2 dictionaries, with the key of each dictionary the fold number.  The value of the dictionary `dict_k_df_X` is the $k^{th}$ partition of the data, and the value of the dictionary `dict_k_s_y` is the corresponding $k^{th}$ target classification.  Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

Note, you can reuse the functions from Section 1. 

In [ ]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5 )

## Q2.3 Define a function that calculates accuracy
The function should accept two series and compare each element for equality.  The accuracy is the number of equal elements divided by the total number of elements.

Test your accuracy function by calling it with the `s_y` loaded from the iris data set and an array of the same length containing all $1$ values. 

In [ ]:
def get_acc( s_1, s_2 ):
    diffDf = pd.DataFrame(s_1 - s_2)
    return (len(diffDf[diffDf.iloc[:, 0] == 0]))/(len(s_1))


In [ ]:
get_acc(s_y,np.ones(len(s_y)))

0.3333333333333333

## Q2.4 Using Nested Cross validation, find the best hyperparameter
Use the [Decision Tree Classifier](https://scikit-learn.org/stable/modules/tree.html#classification) class to build a decision tree inside of a 5-fold cross validation loop.  The partitions you already created in 2.2 will be the outer loop.  In the inside loop you should use 4-fold cross validation (so you don't have to partition _again_) to find the best value for `min_impurity_decrease`.  Use the Gini Index as your impurity measure. 
    Calculate the mean accuracy across the 4 folds of your inner loop for all the candidate `min_impurity_decrease` values, and print the value.  Use the array `np.array([0.1,0.25,0.3,0.4])` as the candidates for the best hyperparameter. If there is a tie (two `min_impurity_decrease` values give the same highest accuracy), choose the lowest `min_impurity_decrease` value. 

For each inner loop, select the best `min_impurity_decrease` and train the outer fold training data on using that value. 

For each of the 5 executions of the inner loop, your output should look something like this:
```
Testing 0.10 min impurity decrease
	Average accuracy over 4 folds is 0.95
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.86
Testing 0.30 min impurity decrease
	Average accuracy over 4 folds is 0.63
Testing 0.40 min impurity decrease
	Average accuracy over 4 folds is 0.27

Best min impurity decrease is 0.1

```

In [ ]:
possible_min_impurity_decrease = np.array([0.1,0.25,0.3,0.4])

# Outer loop
outer_acc = []
for k in dict_k_df_X.keys():
  key_list = list(dict_k_df_X.keys())
  key_list.remove(k)
  dfXTest = dict_k_df_X[k]
  dfYTest = dict_k_s_y[k]

  dfXTrain = pd.DataFrame()
  dfYTrain = pd.DataFrame()
  pm = 0
  for pos_min_impurity in possible_min_impurity_decrease:
    pm = pos_min_impurity
    inner_acc = []
    
    for key in key_list:
      key_list2 = list(dict_k_df_X.keys())
      key_list2.remove(key)
      key_list2.remove(k)
      dfXTestFold = dict_k_df_X[key]
      dfYTestFold = dict_k_s_y[key]

      dfXTrain = pd.concat([dfXTrain, dict_k_df_X[key_list2[0]], dict_k_df_X[key_list2[1]], dict_k_df_X[key_list2[2]]])
      dfYTrain = pd.concat([dfYTrain, dict_k_s_y[key_list2[0]], dict_k_s_y[key_list2[1]], dict_k_s_y[key_list2[2]]])

      clf = tree.DecisionTreeClassifier(criterion="gini", max_depth=2, min_impurity_decrease = pos_min_impurity)
      model = clf.fit(dfXTrain,dfYTrain)
      prediction = model.predict(dfXTestFold)

      inner_acc.append(get_acc( prediction, dfYTestFold ))

      dfXTrain = pd.DataFrame()
      dfYTrain = pd.DataFrame()

  inner_mean = (sum(inner_acc)/len(inner_acc))

  
  print(f"Average accuracy over 4 folds is {inner_mean}")

  outer_acc.append(inner_mean) # make sure and calculate this_acc in your look

print(f"The optimal min impurity decrease is {possible_min_impurity_decrease[outer_acc.index(max(outer_acc))]}")


Average accuracy over 4 folds is 0.3279569892473118
Average accuracy over 4 folds is 0.29610513739545996
Average accuracy over 4 folds is 0.321568100358423
Average accuracy over 4 folds is 0.28378434886499404
Average accuracy over 4 folds is 0.30991338112305855
The optimal min impurity decrease is 0.1


## Q2.5 Show the generalized performance of the classifier 
Show the generalized performance of the classifier by printing the min, max, and mean accuracy of the outer fold test sets.

In [ ]:
print("The min MAE is {:.2f}, the max MAE is {:.2f}, and the mean MAE is {:.2f}".format(min(outer_acc),max(outer_acc),(sum(outer_acc)/len(outer_acc))))

The min MAE is 0.28, the max MAE is 0.33, and the mean MAE is 0.31
